# Usage of Jupyter and Pandas to Analyze Network Data Collected by pyGNMI
The purpose of this doc is to show how to build interactive environment with Jupyter and analyse data from network devices using 

## Step 1. Define hosts to connect
Specify network devices, which you will be connected to in this lab:

In [3]:
HOSTS = [
    {
        "host": "192.168.101.11",
        "port": 57400
    },
    {
        "host": "192.168.101.13",
        "port": 6030
    },
]

## Step 2. Get credentails
Ask user to provide credentials in a secure format

In [4]:
from getpass import getpass

credentails = (input("Provide username > "), getpass("Provide password > "))

## Step 3. Collect information from network devices
Create simple script with pyGNMI to collect `Capabilities` via GNMI and store them in the per-host format

In [5]:
from pygnmi.client import gNMIclient

results = []

for host in HOSTS:
    with gNMIclient(target=(host["host"], host["port"]), username=credentails[0], password=credentails[1], insecure=True) as gconn:
        result = gconn.capabilities()
        results.append((host["host"], result))

Valdiate the collected result

In [6]:
results

[('192.168.101.11',
  {'supported_models': [{'name': 'nokia-conf',
     'organization': 'Nokia',
     'version': '21.7.R1'},
    {'name': 'nokia-state', 'organization': 'Nokia', 'version': '21.7.R1'},
    {'name': 'nokia-li-state', 'organization': 'Nokia', 'version': '21.7.R1'},
    {'name': 'nokia-openconfig-if-ethernet-augments',
     'organization': 'Nokia',
     'version': '21.7.R1'},
    {'name': 'nokia-openconfig-if-ip-augments',
     'organization': 'Nokia',
     'version': '21.7.R1'},
    {'name': 'nokia-openconfig-mpls-augments',
     'organization': 'Nokia',
     'version': '21.7.R1'},
    {'name': 'nokia-openconfig-network-instance-augments',
     'organization': 'Nokia',
     'version': '21.7.R1'},
    {'name': 'nokia-openconfig-transport-types-augments',
     'organization': 'Nokia',
     'version': '21.7.R1'},
    {'name': 'openconfig-aaa',
     'organization': 'OpenConfig working group',
     'version': '0.4.0'},
    {'name': 'openconfig-aaa-types',
     'organization': 

## Step 4. Convert collected data in Pandas Data Frame
Import `pandas` to work with Data Frames

In [7]:
import pandas as pd


### Simple case
First of all, create Data Frame with GNMI versions run per device

In [8]:
gnmi_versions = [{"host": item[0], "gnmi_version": item[1]["gnmi_version"]} for item in results]

pd_gnmi_versions = pd.DataFrame(gnmi_versions)
pd_gnmi_versions

,host,gnmi_version
0,192.168.101.11,0.7.0
1,192.168.101.13,0.7.0


### Complicated case

Create table with supported Protobuf encodings

In [9]:
encodings = [{"host": item[0], **{enc: True for enc in item[1]["supported_encodings"]}} for item in results]

pd_encodings = pd.DataFrame(encodings)
pd_encodings

,host,json,bytes,proto,json_ietf,ascii
0,192.168.101.11,True,True,True,True,NaN
1,192.168.101.13,True,NaN,NaN,True,True


### Medium case

Create table wiht supported YANG modules

In [10]:
yang_modules = [{"host": item[0], **yang_module} for item in results for yang_module in item[1]["supported_models"]]

pd_yang_modules = pd.DataFrame(yang_modules)
pd_yang_modules

,host,name,organization,version
0,192.168.101.11,nokia-conf,Nokia,21.7.R1
1,192.168.101.11,nokia-state,Nokia,21.7.R1
2,192.168.101.11,nokia-li-state,Nokia,21.7.R1
3,192.168.101.11,nokia-openconfig-if-ethernet-augments,Nokia,21.7.R1
4,192.168.101.11,nokia-openconfig-if-ip-augments,Nokia,21.7.R1
...,...,...,...,...
259,192.168.101.13,openconfig-bgp-policy,OpenConfig working group,6.0.2
260,192.168.101.13,openconfig-isis-lsdb-types,OpenConfig working group,0.4.2
261,192.168.101.13,arista-netinst-deviations,"Arista Networks, Inc.",1.0.2
262,192.168.101.13,arista-exp-eos-varp-net-inst,Arista Networks <http://arista.com/>,
